# <center> **ITESO** </center>
# <center> **Final Project Procesamiento de Datos Masivos** </center>
---
## <center> **Spark Post Processing Applications** </center>
## <center> **Real-Time Stock Price Analysis** </center>
---
## <center> **Par de Foraneos** </center>
---
#### <center> **Spring 2025** </center>
---
#### <center> **05/14/2025** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez
**Students**: Eddie, Konrad , Diego, Aaron

## Download necessary Modules

In [ ]:
import findspark
findspark.init()
import os
import glob
from par_de_foraneos.stock_utils import calc_techincal_indicators

## Create Spark Session and Environment


In [7]:
from pyspark.sql import SparkSession

SPARK_SERVER = {'Konrad': '2453c3db49e4',
                'Aaron' : 'a5ab6bdab4b3'}
KAFKA_SERVER = {'Konrad': '4c63f45c41b4:9093',
                'Aaron' : '69b1b3611d90:9093'}
current_user = 'Konrad'

spark = SparkSession.builder \
    .appName("reader-Kafka") \
    .master("spark://{}:7077".format(SPARK_SERVER[current_user])) \
    .config("spark.ui.port","4040") \
    .getOrCreate()
    
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

## FIND PARQUET FILES AND MAKE UNION WITHIN SAME COMPANY

In [8]:
df_s = []

for i in range(4):
    
    files = glob.glob(f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/output{i}/*.parquet")
    #init df for union
    df0 = spark.read.parquet(files[0])

    for f_id in range(1,len(files)):
        df1 = spark.read.parquet(files[f_id])
        df0 = df0.union(df1)
        
    df_s.append(df0)

## REMOVE DUPLICATES CAUSED BY WATERMARKING

In [9]:
for i in range(4):
    a  = df_s[i].orderBy("timestamp", ascending=True).dropDuplicates(["timestamp"])

## CALCULATE INDICATORS AND SAVE AS PARQUET

In [14]:
for i in range(4):
    pandas_df = df_s[i].toPandas()
    custom_resampler = calc_techincal_indicators(pandas_df)
    custom_resampler
    custom_resampler.to_parquet(f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/final_indicator_df_{custom_resampler['company'].iloc[0]}.parquet", index=False)

Calculating technical indicators...
Creating lag features...
Dropping NaN values...
Calculating technical indicators...
Creating lag features...
Dropping NaN values...
Calculating technical indicators...
Creating lag features...
Dropping NaN values...
Calculating technical indicators...
Creating lag features...
Dropping NaN values...


## CLOSE SESSION

In [5]:
sc.stop()